# PART 1 - Video metadata extraction with Amazon Rekognition
In the follow section of the lab, we're going to extract metadata and labels from the video as well as identify the different scenes from the video, all of that via computer vision methods and more precisely via Amazon Rekognition.</br>
</br>
Amazon Rekognition is an AWS services that makes it easy to add image and video analysis to your applications using proven, highly scalable, deep learning technology that requires no machine learning expertise to use. With Amazon Rekognition, you can identify objects, people, text, scenes, and activities in images and videos, as well as detect any inappropriate content. Amazon Rekognition also provides highly accurate facial analysis and facial search capabilities that you can use to detect, analyze, and compare faces for a wide variety of user verification, people counting, and public safety use cases.

https://docs.aws.amazon.com/rekognition/latest/dg/what-is.html

In [1]:
#load stored variable from lab0 notebook
%store -r

In [2]:
import boto3
from botocore.exceptions import ClientError
import os
import time

import logging
import sys
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
log = logging.getLogger('knowledge-graph-logger')

## Scene detection job

Let's start by detecting the different "scenes" or "shots" from the video. This is notably useful to detect fix/black screens.

The below function starts a segment detection job with predefined settings for confidence and coverage notably. We're choosing to identify both "SHOT" (=scenes) and "TECHNICAL_CUE" (e.g. black/fix screen).</br>

In [3]:
rek = boto3.client('rekognition')

def startSegmentDetection(s3_bucket, name):
    min_Technical_Cue_Confidence = 80.0
    min_Shot_Confidence = 80.0
    max_pixel_threshold = 0.1
    min_coverage_percentage = 60

    response = rek.start_segment_detection(
        Video={"S3Object": {"Bucket": s3_bucket, "Name": name}},
        NotificationChannel={
            "RoleArn": role_arn,
            "SNSTopicArn": sns_topic_arn,
        },
        SegmentTypes=["TECHNICAL_CUE", "SHOT"],
        Filters={
            "TechnicalCueFilter": {
                "MinSegmentConfidence": min_Technical_Cue_Confidence,
            },
            "ShotFilter": {"MinSegmentConfidence": min_Shot_Confidence},
        }
    )
    return response
    
segment_detection_job = startSegmentDetection(bucket, os.path.join(s3_video_input_path, video_file))

In [4]:
segment_detection_job_id = segment_detection_job['JobId']
print(f"Job Id: {segment_detection_job_id}")

#Grab the segment detection response
SegmentDetectionOutput = rek.get_segment_detection(
    JobId=segment_detection_job_id
)

#Determine the state. If the job is still processing we will wait a bit and check again
while(SegmentDetectionOutput['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    SegmentDetectionOutput = rek.get_segment_detection(
    JobId=segment_detection_job_id)
    
#Once the job is no longer in progress we will proceed
print(SegmentDetectionOutput['JobStatus'])

Job Id: 3533cce3348f7faf05f5e8c30d0e5f4170239d93f7d48233dc3e10ef9f64ce68
..........SUCCEEDED


Let's have a look at the output from the job

In [5]:
SegmentDetectionOutput['VideoMetadata']

[{'Codec': 'h264',
  'DurationMillis': 278028,
  'Format': 'QuickTime / MOV',
  'FrameRate': 23.976024627685547,
  'FrameHeight': 1080,
  'FrameWidth': 1920,
  'ColorRange': 'LIMITED'}]

In [6]:
SegmentDetectionOutput['AudioMetadata']

[{'Codec': 'aac',
  'DurationMillis': 278082,
  'SampleRate': 44100,
  'NumberOfChannels': 2}]

In [7]:
#picking a random scene. note that it will either be of type TECHNICAL_CUE or SHOT 
SegmentDetectionOutput['Segments'][:5]

[{'Type': 'TECHNICAL_CUE',
  'StartTimestampMillis': 0,
  'EndTimestampMillis': 277944,
  'DurationMillis': 277944,
  'StartTimecodeSMPTE': '00:00:00:00',
  'EndTimecodeSMPTE': '00:04:37:16',
  'DurationSMPTE': '00:04:37:16',
  'TechnicalCueSegment': {'Type': 'Content', 'Confidence': 100.0},
  'StartFrameNumber': 0,
  'EndFrameNumber': 6664,
  'DurationFrames': 6664},
 {'Type': 'TECHNICAL_CUE',
  'StartTimestampMillis': 21980,
  'EndTimestampMillis': 26901,
  'DurationMillis': 4921,
  'StartTimecodeSMPTE': '00:00:21:23',
  'EndTimecodeSMPTE': '00:00:26:21',
  'DurationSMPTE': '00:00:04:22',
  'TechnicalCueSegment': {'Type': 'OpeningCredits', 'Confidence': 100.0},
  'StartFrameNumber': 527,
  'EndFrameNumber': 645,
  'DurationFrames': 118},
 {'Type': 'TECHNICAL_CUE',
  'StartTimestampMillis': 252835,
  'EndTimestampMillis': 262595,
  'DurationMillis': 9760,
  'StartTimecodeSMPTE': '00:04:12:14',
  'EndTimecodeSMPTE': '00:04:22:08',
  'DurationSMPTE': '00:00:09:18',
  'TechnicalCueSegmen

Let's park segment detection for the moment. we'll use those metadata in the Neptune related notebook when storing our data into the graph.

## Label detection in the video
We're now using a different functionality of Amazon Rekognition: Label detection.</br>
By running this job on the video, Rekognition will automatically labels objects, concepts, scenes, and actions in your images, and provides an associated confidence score.

In [8]:
#method starting a individual rekognition labeling job
def start_label_detection(s3_bucket, name, roleArn, sns_topic_arn):
    response = rek.start_label_detection(
        Video={"S3Object": {"Bucket": s3_bucket, "Name": name}},
        NotificationChannel={
            "RoleArn": roleArn,
            "SNSTopicArn": sns_topic_arn,
        }
    )
    return response

Launching the label detection job.

In [9]:
#launching it on our test video.
label_detection_response = start_label_detection(bucket, os.path.join(s3_video_input_path, video_file), role_arn, sns_topic_arn)

Monitoring when the job is finished

In [10]:
label_detection_job_id = label_detection_response['JobId']
print(f"Job Id: {label_detection_job_id}")

#Grab the segment detection response
LabelDetectionOutput = rek.get_label_detection(
    JobId=label_detection_job_id
)

#Determine the state. If the job is still processing we will wait a bit and check again
while(LabelDetectionOutput['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    LabelDetectionOutput = rek.get_label_detection(
    JobId=label_detection_job_id)
    
#Once the job is no longer in progress we will proceed
print(LabelDetectionOutput['JobStatus'])

Job Id: abe2bbca1a852ae8d3dc904652bcc5e15d8686bc2c8fb31fd6ab38dfa0e76e4d
.....................SUCCEEDED


Let's have a look at the output from the label detection job

In [11]:
LabelDetectionOutput['Labels'][:5]

[{'Timestamp': 0,
  'Label': {'Name': 'Apparel',
   'Confidence': 89.63540649414062,
   'Instances': [],
   'Parents': []}},
 {'Timestamp': 0,
  'Label': {'Name': 'Blossom',
   'Confidence': 62.90474319458008,
   'Instances': [],
   'Parents': [{'Name': 'Plant'}]}},
 {'Timestamp': 0,
  'Label': {'Name': 'Clothing',
   'Confidence': 89.63540649414062,
   'Instances': [],
   'Parents': []}},
 {'Timestamp': 0,
  'Label': {'Name': 'Female',
   'Confidence': 54.17314910888672,
   'Instances': [],
   'Parents': [{'Name': 'Person'}]}},
 {'Timestamp': 0,
  'Label': {'Name': 'Flower',
   'Confidence': 62.90474319458008,
   'Instances': [],
   'Parents': [{'Name': 'Plant'}]}}]

Again, let's park the label detection for the moment and export/store the outputs of those 2 jobs so that we can use it in part 3 for when we create our Graph.

In [12]:
%store SegmentDetectionOutput
%store LabelDetectionOutput

Stored 'SegmentDetectionOutput' (dict)
Stored 'LabelDetectionOutput' (dict)
